In [43]:
#!pip install --upgrade git+https://github.com/EmGarr/kerod.git

In [44]:
#%tensorflow_version 2.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
# if device_name != '/device:GPU:0':
#   raise SystemError('GPU device not found')
# print('Found GPU at: {}'.format(device_name))

In [45]:
import tensorflow as tf
x = tf.constant([1,2,1,2,3,4,2])
x = tf.expand_dims(x, axis=0)
x = tf.tile(x, [x.shape[1], 1])
x_ = tf.transpose(x)
Y = tf.where(tf.equal(x,x_), tf.ones_like(x), tf.zeros_like(x))
Y

<tf.Tensor: shape=(7, 7), dtype=int32, numpy=
array([[1, 0, 1, 0, 0, 0, 0],
       [0, 1, 0, 1, 0, 0, 1],
       [1, 0, 1, 0, 0, 0, 0],
       [0, 1, 0, 1, 0, 0, 1],
       [0, 0, 0, 0, 1, 0, 0],
       [0, 0, 0, 0, 0, 1, 0],
       [0, 1, 0, 1, 0, 0, 1]], dtype=int32)>

In [46]:
import warnings
warnings.filterwarnings('ignore')

# Download COCO/2017

Download and preprocess COCO/2017 to the following format (required by od networks):

```python
dataset = {
        'images' : A tensor of float32 and shape [1, height, widht, 3],
        'images_info': A tensor of float32 and shape [1, 2] ,
        'bbox': A tensor of float32 and shape [1, num_boxes, 4],
        'labels': A tensor of int32 and shape [1, num_boxes],
        'num_boxes': A tensor of int32 and shape [1, 1],
        'weights': A tensor of float32 and shape [1, num_boxes]
    }
```

If you need to download the dataset in a specific directory you can use the argument `data_dir` of `tfds.load`.

In [47]:
import functools
import tensorflow as tf
import tensorflow_datasets as tfds
from   tensorflow.keras.utils import to_categorical
import  matplotlib.pyplot as plt
from tensorflow.keras import layers
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint

# from kerod.dataset.preprocessing import preprocess, expand_dims_for_single_batch
# from kerod.core.standard_fields import BoxField


In [48]:
def my_read_image(image_path,label):
    image = tf.io.read_file(image_path)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, (299, 299))
    image = tf.cast(image, tf.float32)
    image /= 127.5
    image -= 1.
    return image, image


In [49]:
# ds_train, ds_info = tfds.load(name="coco/2017", split="train", data_dir='/home/suphale/coco', shuffle_files=True, with_info=True)


In [50]:
class BoxField:
    BOXES = 'bbox'
    KEYPOINTS = 'keypoints'
    LABELS = 'label'
    MASKS = 'masks'
    NUM_BOXES = 'num_boxes'
    SCORES = 'scores'
    WEIGHTS = 'weights'


class DatasetField:
    IMAGES = 'images'
    IMAGES_INFO = 'images_information'
    IMAGES_PMASK = 'images_padding_mask'

def my_preprocess(inputs):
    # tf.print("my_preprocess: type(input) %s" % (str(type(inputs))))
    # print("my_preprocess: type(x) %s" % (str(type(x))))
    # tf.print(inputs['image/filename'])
    image = inputs['image']
    image = tf.image.resize(image, (299, 299))
    image = tf.cast(image, tf.float32)
    image /= 127.5
    image -= 1.

    targets = inputs['objects']

    image_information = tf.cast(tf.shape(image)[:2], dtype=tf.float32)

    inputs = {DatasetField.IMAGES: image, DatasetField.IMAGES_INFO: image_information}

    # ground_truths = {
    #     BoxField.BOXES: targets[BoxField.BOXES] * tf.tile(image_information[tf.newaxis], [1, 2]),
    #     BoxField.LABELS: tf.cast(targets[BoxField.LABELS], tf.int32),
    #     BoxField.NUM_BOXES: tf.shape(targets[BoxField.LABELS]),
    #     BoxField.WEIGHTS: tf.fill(tf.shape(targets[BoxField.LABELS]), 1.0)
    # }
    ground_truths = tf.cast(targets[BoxField.LABELS], tf.int32)
    # tf.print(tf.shape(ground_truths))
    ground_truths = tf.one_hot(ground_truths, depth=80, dtype=tf.int32)
    # tf.print(tf.shape(ground_truths))
    ground_truths = tf.reduce_sum(ground_truths, 0)
    # tf.print(tf.shape(ground_truths))
    return image, ground_truths
    # return inputs

def expand_dims_for_single_batch(image, ground_truths):

    # tf.print(tf.shape(image))
    image = tf.expand_dims(image, axis=0)
    ground_truths = tf.expand_dims(ground_truths, axis=0)
    return image, ground_truths



In [51]:
# ds_train = ds_train.map(functools.partial(my_preprocess), num_parallel_calls=tf.data.experimental.AUTOTUNE)
# sample_img_batch, sample_cap_batch = next(iter(ds_train))
# plt.imshow(sample_img_batch)

In [52]:
# tf.argmax(sample_cap_batch[0])
# sample_cap_batch

In [53]:
data_dir='/home/suphale/coco'

ds_train, ds_info = tfds.load(name="coco/2017", split="train", data_dir=data_dir, shuffle_files=True, download=False, with_info=True)
ds_train = ds_train.map(functools.partial(my_preprocess), num_parallel_calls=tf.data.experimental.AUTOTUNE)
# Filter example with no boxes after preprocessing
# ds_train =  ds_train.filter(lambda x, y: tf.shape(y[BoxField.BOXES])[0] > 1)
ds_train = ds_train.map(expand_dims_for_single_batch, num_parallel_calls=tf.data.experimental.AUTOTUNE)
ds_train = ds_train.prefetch(tf.data.experimental.AUTOTUNE)

ds_test = tfds.load(name="coco/2017", split="validation", data_dir=data_dir, shuffle_files=False, download=False)
ds_test = ds_test.map(
    functools.partial(my_preprocess),
    num_parallel_calls=tf.data.experimental.AUTOTUNE)
# Filter example with no boxes after preprocessing
# ds_test =  ds_test.filter(lambda x, y: tf.shape(y[BoxField.BOXES])[0] > 1)
ds_test = ds_test.map(expand_dims_for_single_batch, num_parallel_calls=tf.data.experimental.AUTOTUNE)
ds_test = ds_test.prefetch(tf.data.experimental.AUTOTUNE)


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


In [54]:
ds_info

tfds.core.DatasetInfo(
    name='coco',
    full_name='coco/2017/1.1.0',
    description="""
    COCO is a large-scale object detection, segmentation, and
    captioning dataset.
    
    Note:
     * Some images from the train and validation sets don't have annotations.
     * Coco 2014 and 2017 uses the same images, but different train/val/test splits
     * The test split don't have any annotations (only images).
     * Coco defines 91 classes but the data only uses 80 classes.
     * Panotptic annotations defines defines 200 classes but only uses 133.
    """,
    config_description="""
    
    This version contains images, bounding boxes and labels for the 2017 version.
    
    """,
    homepage='http://cocodataset.org/#home',
    data_path='/home/suphale/coco/coco/2017/1.1.0',
    download_size=25.20 GiB,
    dataset_size=24.98 GiB,
    features=FeaturesDict({
        'image': Image(shape=(None, None, 3), dtype=tf.uint8),
        'image/filename': Text(shape=(), dtype=tf.string

# Load and train the network


In [55]:
# sample_img_batch, sample_cap_batch = next(iter(ds_train))
# # plt.imshow(sample_img_batch)
# tf.shape(sample_img_batch)


In [56]:
# sample_cap_batch


In [57]:
# classes = ds_info.features['objects']['label'].names
# num_classes = len(classes)
# print(num_classes)

In [58]:
import numpy as np
import functools
from tensorflow.keras import backend as K
import itertools

def w_categorical_crossentropy(y_true, y_pred, weights):
    tf.print("--Snehal---------------------------------------------------------------------->")
    nb_cl = len(weights)
    final_mask = K.zeros_like(y_pred[:, 0])
    y_pred_max = K.max(y_pred, axis=1)
    y_pred_max = K.expand_dims(y_pred_max, 1)
    y_pred_max_mat = K.equal(y_pred, y_pred_max)
    for c_p, c_t in itertools.product(range(nb_cl), range(nb_cl)):
        final_mask += (K.cast(weights[c_t, c_p],K.floatx()) * K.cast(y_pred_max_mat[:, c_p] ,K.floatx())* K.cast(y_true[:, c_t],K.floatx()))
        
    tf.print("Snehal")
    tf.print(tf.shape(y_pred))
    tf.print(tf.shape(y_true))
    tf.print(tf.shape(final_mask))
    return K.categorical_crossentropy(y_pred, y_true) * final_mask

w_array = np.ones((2,2))
w_array[1,0] = 1.2
ncce = functools.partial(w_categorical_crossentropy, weights=w_array)
ncce.__name__ = "w_categorical_crossentropy"
# ncce.__module__ = my_other_view.__module__

In [59]:
#Save the model after every epoch.
# mc_top = ModelCheckpoint(top_layers_checkpoint_path, monitor='val_acc', verbose=0, save_best_only=True, save_weights_only=False, mode='auto', period=1)

#Save the TensorBoard logs. histogram_freq was 1 (gave errors) and now is 0. write_images was True (read that this is heavy) and now is False
# tb = TensorBoard(log_dir='./logs', histogram_freq=0, write_graph=False, write_images=False)


In [60]:
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D

image_model = tf.keras.applications.InceptionV3(include_top=False,weights='imagenet')

N_LABELS = 80

# write code here to get the input of the image_model
new_input = image_model.input 
# write code here to get the output of the image_model
hidden_layer = image_model.output 

#build the final model using both input & output layer
image_features_extract_model = tf.keras.Model(inputs=new_input, outputs=hidden_layer)

x = image_model.output
x = GlobalAveragePooling2D()(x)
# let's add a fully-connected layer
x = tf.keras.layers.Dense(1024, activation='relu')(x)
# and a logistic layer -- let's say we have 200 classes
predictions = tf.keras.layers.Dense(80, activation='softmax')(x)

# this is the model we will train
model = tf.keras.Model(inputs=image_model.input, outputs=predictions)

for layer in image_model.layers:
    layer.trainable = False

# model.compile(optimizer='rmsprop', loss=ncce, metrics=['accuracy'])
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
# model.compile(optimizer='rmsprop', loss='categorical_crossentropy')



In [61]:
# model.summary()

In [62]:

callbacks = [
    TensorBoard(),
    ModelCheckpoint('checkpoints/')
]

model.fit(ds_train, validation_data=ds_test, epochs=12, callbacks=callbacks)


Epoch 1/12
    18/118287 [..............................] - ETA: 42:22:36 - loss: 161.9847 - accuracy: 0.1472

In [ ]:
# from kerod.core.standard_fields import BoxField
# from kerod.core.learning_rate_schedule import LearningRateScheduler
# from kerod.model import factory

# from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint

# # Number of classes of COCO
# classes = ds_info.features['objects']['label'].names
# num_classes = len(classes)

# model_faster_rcnn = factory.build_model(num_classes)
# base_lr = 0.02
# optimizer = tf.keras.optimizers.SGD(learning_rate=base_lr, momentum=0.9)
# model_faster_rcnn.compile(optimizer=optimizer, loss=None)

# #The numbering of epochs (LearningRateScheduler) starts at 0.
# # Which means the decrease will happens on the epoch 9:
# #(8 + 1: numbering of fit logging starts at 1)
# callbacks = [
#     TensorBoard(),
#     ModelCheckpoint('checkpoints/')
# ]

# model_faster_rcnn.fit(ds_train, validation_data=ds_test, epochs=12, callbacks=callbacks)

In [ ]:
# Save the weights for the serving
model.save_weights('final_weights.h5')

In [ ]:
# Export a saved model for serving purposes
model.export_for_serving('serving')

## Visualisation of few images

In [ ]:
from od.utils.drawing import BoxDrawer

drawer = BoxDrawer(classes)

for i, example in enumerate(ds_val):
    inputs, ground_truths = example
    out = model.predict_on_batch(inputs)
    boxes, scores, labels, valid_detections = out
    # Will draw the results
    drawer(
        inputs['images'],
        boxes,
        scores=scores,
        labels=labels,
        num_valid_detections=valid_detections
    )
    if i == 5:
        break

## Tensorboard

In [ ]:
# Load TENSORBOARD
%load_ext tensorboard
# Start TENSORBOARD
%tensorboard --logdir logs

## Coco evaluation


### Load the dataset

In [ ]:
import tensorflow_datasets as tfds

ds_val, ds_info = tfds.load(name="coco/2017", split="validation", shuffle_files=False, with_info=True)
# category_ids basicaly map the index 0 the id
# e.g: 0 -> 1, 2 -> 3, 79 -> 90
category_ids = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 27, 28, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 67, 70, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 84, 85, 86, 87, 88, 89, 90]

### Super dirty but the evaluation works

1. perform the analysis

In [ ]:
import json
import numpy as np
import tensorflow as tf

from kerod.core.standard_fields import DatasetField, BoxField             
from kerod.core.box_ops import convert_to_center_coordinates              
from kerod.dataset.preprocessing import resize_to_min_dim                 
                                                                       
results = []                                                           
                                                                       
for i, example in enumerate(ds_val): 
    print(i)
    # preprocess image
    image = example['image'][:, :, ::-1]
    image = resize_to_min_dim(image, 800.0, 1333.0)         
    image_information = tf.cast(tf.shape(image)[:2], dtype=tf.float32) 
    inputs = {
        'images': tf.expand_dims(image, axis=0),
        'images_information':tf.expand_dims(image_information, axis=0)
    }
                                                                 
    # predict                                                          
    boxes, scores, labels, valid_detections = model_faster_rcnn.predict_on_batch(inputs)
                                                                       
    # Post processing and append to coco results                       
    bbox = boxes[0] * tf.tile(
        tf.expand_dims(tf.cast(example['image'].shape[:2], tf.float32), axis=0),
        [1, 2])                   
    scores = scores[0]                                           
    labels = labels[0]                                           
    for i in range(valid_detections[0]):
        # Convert from [y_min, x_min, y_max, x_max] to coco format [x_min, y_min, w, h]
        sbox = bbox[i].numpy()
        sbox = [sbox[1], sbox[0], sbox[3] - sbox[1], sbox[2] - sbox[0]]
        res = {                                                        
                'image_id': int(example['image/id']),                       
                'category_id': category_ids[int(labels[i])],           
                'bbox': [round(float(c), 4) for c in sbox],
                'score': round(float(scores[i]), 4),                     
            }                                                          
        results.append(res)                                            
                                                   
                                                                       
with open('coco_results.json', 'w') as f:                              
    json.dump(results, f)

2. install the coco library to compute the performances

In [ ]:
!wget http://images.cocodataset.org/annotations/annotations_trainval2017.zip
!unzip annotations_trainval2017.zip
!pip install 'git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI'

3. compute the performances

In [ ]:
from pycocotools.coco import COCO
from pycocotools.cocoeval import COCOeval

with open('coco_results_corrected.json', 'r') as f:                              
    results = json.load(f)
coco = COCO('./annotations/instances_val2017.json')
ret = {}

cocoDt = coco.loadRes(results)
cocoEval = COCOeval(coco, cocoDt, 'bbox')
cocoEval.evaluate()
cocoEval.accumulate()
cocoEval.summarize()